In [ ]:
from transformers import AutoTokenizer, GenerationConfig, AutoModelForCausalLM
import torch

In [ ]:
model_path = "../../../GROUP/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path, 
    torch_dtype=torch.bfloat16,
    device_map="auto",
    use_auth_token=True
)

In [ ]:
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# Generation config set to default values
config = GenerationConfig(
    max_new_tokens=20,
    do_sample=False,
    num_beams=1,
    temperature=1.0,
    top_k=50, 
    top_p=1.0,
)
    

outputs = model.generate(
    input_ids,
    eos_token_id=terminators,
    generation_config=config
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))